In [107]:
#################################
######### CHANGE THESE: #########
#################################

#get token from https://developer.spotify.com/console/post-playlist-tracks/
TOKEN = 'BQA9xzG29-Wi8PLP5Z1YSSmDOAo3Vxke2OJPscQz27HYC1j-3phPTI1-3BhCl-MgmD65Ct81Vp5q5eRNE_P3dXDImRxAWAR9-Rys098ACnfffxKH7O-gxR15HqdAHqsJNoPA5yjua75-kEqYimTZVvvBSHtuEiBdmRLUJSuS9zte3Er9gRrNZkC84kg0cV3gdoSu0-pmhEq-611CGo9WfxFzywXvKCsfWqoSeyn_gxKduYlsP9jCw2TGiIodUysgPgo5EFDTRSRBWwhM0oJf5w'
USER = 'jae94lee' #replace with your own spotify user id

#authentification
cid ="39bc11236f6a4bd79c9eadcdbce37c4e" 
secret = "9ee75c639b394b54b7bf9ac5cee6a6f9"



########### Imports ###########
import json
import requests
from math import ceil
import pandas as pd
from pandas.io.json import json_normalize
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
from datetime import datetime
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)


client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
headers = {'Authorization': "Bearer {}".format(TOKEN)}



##################################################################
########## EXTRACT DATA FROM USER'S EXISTING PLAYLISTS ###########
##################################################################

########## Get playlists ###########
def get_playlists():
    try:
        #50 is the max
        r5 = requests.get("https://api.spotify.com/v1/users/" + USER + "/playlists?offset=1&limit=50", headers=headers)
    except:
        pass
                
    pl = json.loads(r5.text)
    playlists_id_list = []    
    for i in range (0,50):
        playlists = pl['items'][i]['id']
        playlists_id_list.append(playlists)
        
    playlists_id_list_new = ','.join(str(i) for i in playlists_id_list)
    return playlists_id_list


########### Get audio features of each track id ###########
def get_audio_features(song_id_list):
    audio_features_url = f"https://api.spotify.com/v1/audio-features?ids="

    try:
        r2 = requests.get(audio_features_url + song_id_list, headers=headers)
    except:
        pass

    audio_features = json.loads(r2.text)['audio_features']
    audio_features_removed_none = [item for item in audio_features if item != None]
    x = pd.DataFrame.from_records(audio_features_removed_none)
    return x


########### Get Time and Key Signatures and mode ##########
def get_key_and_time_signature(song_id_list):
    
    #pitch class dict
    pitch_class = list(range(0,12))
    tone = ['C','C#','D','Eb','E','F','F#','G','G#','A','Bb','B']
    pitch_class_dict = {pitch_class[i]:tone[i] for i in range(len(pitch_class))}

    #mode dict
    mode_dict = {0:'minor', 1:'major', -1:'no_result'}
    
    features_df = get_audio_features(song_id_list)
    kts_df = features_df[['id', 'key', 'mode', 'time_signature']]  
    kts_df['key'] = kts_df['key'].replace(pitch_class_dict)
    kts_df['mode'] = kts_df['mode'].replace(mode_dict)
    kts_df.set_index('id')
    return kts_df

########### Get Other Attributes ##########
def get_other_features(song_id_list):
    #danceability = based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. 
    #A value of 0.0 is least danceable and 1.0 is most danceable.

    #energy = [0.0 to 1.0] and represents a perceptual measure of intensity and activity. #Typically, energetic tracks feel fast, loud, and noisy. 
    #For example, death metal has high energy, while a Bach prelude scores low on the scale. 
    #Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.

    #valence = A measure [0.0 to 1.0] describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).
    
    #loudness = The overall loudness of a track in decibels (dB). 
    #Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. 
    #Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). 
    #Values typically range between -60 and 0 db.
    features_df = get_audio_features(song_id_list)
    select_features_df = features_df[['id', 'danceability', 'energy', 'loudness', 'tempo', 'valence']] 
    select_features_df.set_index('id')
    return select_features_df


########### Get playlist's track ids ############
def get_playlist_tracks(playlist_id):
    playlist_tracks_url = f"https://api.spotify.com/v1/playlists/"        
        
    try:
        r3 = requests.get(playlist_tracks_url + playlist_id + f"/tracks", headers=headers)
        playlist_tracks = json.loads(r3.text)
        pt = playlist_tracks['items']
        pt_df =  pd.json_normalize(pt)
    except:
        pass
    
    #get artists and id
    artist_list = []
    song_id_list = []
    for i in range(0, len(pt)):
        song = pt[i]['track']
        if song is None:
            pass     
        else:
            artist = song['artists'][0]['name']
            artist_list.append(artist)
            song_id = song['id']
            song_id_list.append(song_id)

    artists_and_id = pd.DataFrame()
    artists_and_id['id'] = song_id_list
    artists_and_id['artist'] = artist_list
    artists_and_id.set_index('id', inplace=True) 
        
    #get name and id
    if pt_df.empty:
        pass
    else:
        name_and_id = pt_df[['track.id','track.name']]
        name_and_id.rename(columns={"track.id": "id", "track.name": "name"}, inplace=True)
        name_and_id.set_index('id', inplace=True)

        tracks_id_list = [] 
        for i in range(0, len(name_and_id)):
            tracks = pt[i]['track']
            if tracks is None:
                pass
            else:
                new = tracks['id']
                tracks_id_list.append(new)

        tracks_id_list_new = ','.join(str(i) for i in tracks_id_list)

        #get audio features of each track id
        x = get_key_and_time_signature(tracks_id_list_new)
        key_time_sig = x.set_index('id')

        y = get_other_features(tracks_id_list_new)
        other_features = y.set_index('id')

        #combine audio features with name/artist/id    
        nameartist = name_and_id.join(artists_and_id)
        #final = nameartist.join(key_time_sig)
        final = nameartist.join(other_features)
        return final



################################################
###################### RUN #####################
################################################

#get the playlist ids
playlists_id_list = get_playlists()
print(playlists_id_list)

#for each playlist, make a small dataframe and add to agg_df
agg_df = pd.DataFrame()

for count, playlist_id in enumerate(playlists_id_list):
    one_playlist_df = get_playlist_tracks(playlist_id)   
    agg_df = agg_df.append(one_playlist_df)

########## Export to csv ##########
#agg_df.to_csv('spotify_key_time_signature_output.csv')
agg_df.to_csv('spotify_workout_music_features_output.csv')


##############################################################
############### Create Playlist from track ids ###############
##############################################################


########## Create new playlist ##########
def create_new_playlist(playlist_name, description, public):
    new_playlist_url = f"https://api.spotify.com/v1/users/{USER}/playlists"
    request_body = json.dumps({
              "name": f'{playlist_name}',
              "description": f'{description}',
              "public": f'{public}'
            })
    headers = {"Content-Type":"application/json", 'Authorization': "Bearer {}".format(TOKEN)}
    try: 
        response = requests.post(new_playlist_url, data = request_body, headers=headers)
        return response
    except:
        pass
    


########## Populate the playlist with manipulated list of track ids ##########
def populate_playlist(id_list, playlist_name, description, public):  
    response = create_new_playlist(playlist_name, description, public)
    
    pre_uris = f"https://api.spotify.com/v1/playlists/"
    headers = {'Authorization': "Bearer {}".format(TOKEN)}
    playlist_id = response.json()['id']

    # get track ids in batches of 10 to avoid 414 response
    new_ids = ["spotify:track:" + item for item in id_list]
    length = len(new_ids)
    i = 0

    while i+10 < length:
        ids_10 = ','.join(str(x) for x in new_ids[i:i+10])

        uris = pre_uris + playlist_id + '/tracks?uris=' + ids_10
        try:   
            response = requests.post(uris, headers=headers)
        except:
            pass
        i = i+10


KeyError: 'items'

In [105]:
########## RUN TO CREATE PLAYLISTS ##########

pre_uris = f"https://api.spotify.com/v1/playlists/"
headers = {'Authorization': "Bearer {}".format(TOKEN)}
    
    
########## Manipulation. Do whatever you want. ##########
#'danceability', 'energy', 'loudness', 'tempo', 'valence'

#Bb minor key only
#dff = agg_df[(agg_df['key'] == 'Bb') & (agg_df['mode'] == 'minor')]

TEMPO = 150
desc = 'agg_df[''danceability''] >= 0.7, agg_df[''energy''] >= 0.7'
features = [agg_df['danceability'] >= 0.7, agg_df['energy'] >= 0.7]
dff = agg_df[(features[0]) & (features[1]) & (agg_df['tempo'] >= TEMPO-(TEMPO/10)) & (agg_df['tempo'] <= TEMPO+(TEMPO/10))]

print(dff.shape)

dff.head(5)


(19, 7)


,name,artist,danceability,energy,loudness,tempo,valence
id,,,,,,,
2d8JP84HNLKhmd6IYOoupQ,Trap Queen,Fetty Wap,0.746,0.873,-3.803,148.075,0.817
1A7QIlETJSCFXzyp3cSbak,7/11,Beyoncé,0.747,0.705,-5.137,136.024,0.560
3GzjG6Qah8cplpj5YAURLK,Say Yeah,Wiz Khalifa,0.751,0.806,-6.100,142.015,0.685
6RtPijgfPKROxEzTHNRiDp,Rude,MAGIC!,0.773,0.758,-4.993,144.033,0.925
5oeJbBuZwo04t4AAIaYh2S,zZz,DROELOE,0.726,0.713,-8.425,150.066,0.488


In [106]:
#create deduped list of track ids to pass
ids = dff.index.drop_duplicates(keep='first')
id_list = ids.tolist()

PLAYLIST_NAME = f'Workout Playlist {TEMPO} BPM'
DESCRIPTION = f'Created using custom code. {desc}'
PUBLIC = False

populate_playlist(ids, PLAYLIST_NAME, DESCRIPTION, PUBLIC)

KeyError: 'id'